In [392]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

In [393]:
data = pd.read_csv(r'C:\Users\YONSAI\Desktop\Python_Studying\통계\data\realfinal.csv', encoding = 'cp949')
data.head()

,city county,target,ap,grow_fi,grow_fsd,hos,popu_pgr,popu_poep,popu_mip,popu_om,...,rsrs_tr,rsrs_fi,rsrs_ci,rsrs_li,rsrs_su,rsrs_in,gre,npptp,hbtp,s10
0,asan city,1,15000,40.9,56.9,5.2,0.86,12.24,3851,3566,...,3.0,2.0,4.0,3.0,3.0,3.0,57.58,1.8,11.8,16.2
1,boeun-gun,0,7700,7.7,64.6,15.3,-2.15,31.76,282,301,...,2.0,2.0,2.0,3.0,3.0,3.0,61.25,2.1,21.2,15.2
2,boryeong city,1,11267,20.4,67.2,8.7,-0.69,23.76,1030,1187,...,4.0,5.0,2.0,5.0,5.0,4.0,54.44,1.7,10.6,15.8
3,buyeo-gun,0,8233,14.3,57.1,0.2,-1.83,31.72,563,597,...,4.0,5.0,3.0,2.0,4.0,4.0,89.92,2.0,17.0,28.5
4,cheonan city,1,23267,47.6,68.0,18.4,0.96,9.86,9047,8580,...,3.0,2.0,4.0,3.0,3.0,2.0,64.13,3.3,14.5,15.6


In [394]:
# 행 제거
data.drop(index = [8,41], axis = 0, inplace = True)

# 열 제거
data.drop(columns = ['city county'], axis = 1, inplace = True)
data.head()

,target,ap,grow_fi,grow_fsd,hos,popu_pgr,popu_poep,popu_mip,popu_om,cpp,...,rsrs_tr,rsrs_fi,rsrs_ci,rsrs_li,rsrs_su,rsrs_in,gre,npptp,hbtp,s10
0,1,15000,40.9,56.9,5.2,0.86,12.24,3851,3566,31.3,...,3.0,2.0,4.0,3.0,3.0,3.0,57.58,1.8,11.8,16.2
1,0,7700,7.7,64.6,15.3,-2.15,31.76,282,301,34.0,...,2.0,2.0,2.0,3.0,3.0,3.0,61.25,2.1,21.2,15.2
2,1,11267,20.4,67.2,8.7,-0.69,23.76,1030,1187,31.0,...,4.0,5.0,2.0,5.0,5.0,4.0,54.44,1.7,10.6,15.8
3,0,8233,14.3,57.1,0.2,-1.83,31.72,563,597,32.4,...,4.0,5.0,3.0,2.0,4.0,4.0,89.92,2.0,17.0,28.5
4,1,23267,47.6,68.0,18.4,0.96,9.86,9047,8580,33.6,...,3.0,2.0,4.0,3.0,3.0,2.0,64.13,3.3,14.5,15.6


In [395]:
# NaN 값을 중간값으로 대체
data = data.fillna(data.median())
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 0 to 65
Data columns (total 48 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   target          64 non-null     int64  
 1   ap              64 non-null     int64  
 2   grow_fi         64 non-null     float64
 3   grow_fsd        64 non-null     float64
 4   hos             64 non-null     float64
 5   popu_pgr        64 non-null     float64
 6   popu_poep       64 non-null     float64
 7   popu_mip        64 non-null     int64  
 8   popu_om         64 non-null     int64  
 9   cpp             64 non-null     float64
 10  housing_noh     64 non-null     int64  
 11  lta             64 non-null     float64
 12  ahr             64 non-null     float64
 13  v               64 non-null     float64
 14  wsc             64 non-null     float64
 15  scr             64 non-null     float64
 16  p               64 non-null     float64
 17  ic              64 non-null     float

In [396]:
# 데이터프레임에서 상관 계수 계산
corr_matrix = data.corr()

In [397]:
data.columns

Index(['target', 'ap', 'grow_fi', 'grow_fsd', 'hos', 'popu_pgr', 'popu_poep',
       'popu_mip', 'popu_om', 'cpp', 'housing_noh', 'lta', 'ahr', 'v', 'wsc',
       'scr', 'p', 'ic', 'ahs', 'psap', 'sosang', 'so', 'jungso', 'jung',
       'dae', 'er_sgg_man', 'er_sgg_woman', 'lfpr_sgg_man', 'lfpr_sgg_woman',
       'bid', 'wgr', 'clc', 'spfm', 'pspti', 'cpp.1', 'pls', 'pol', 'fir',
       'rsrs_tr', 'rsrs_fi', 'rsrs_ci', 'rsrs_li', 'rsrs_su', 'rsrs_in', 'gre',
       'npptp', 'hbtp', 's10'],
      dtype='object')

In [398]:
# 전체 데이터셋을 X로 준비
X = data.drop(['target', 'sosang', 'er_sgg_man', 'jungso', 'lfpr_sgg_man', 'popu_poep', 'housing_noh', 'p', 'lfpr_sgg_woman',
               'popu_mip', 'jung', 'gre', 'wsc', 'scr', 'pol', 'grow_fsd', 'clc', 'cpp', 'pspti', 'popu_om', 'grow_fi',
               'rsrs_li', 'rsrs_su', 'pls', 'v', 'fir', 'rsrs_tr', 'rsrs_ci', 'npptp', 'spfm', 'rsrs_in', 'ahr', 'rsrs_fi', 'ahs'], axis=1)  # 종속 변수를 제외한 독립 변수들

# 각 변수의 VIF 값을 계산
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# 결과 출력
print(vif)

         Feature        VIF
0             ap  15.139469
1            hos   5.076138
2       popu_pgr   3.941454
3            lta   8.819065
4             ic   5.179201
5           psap   9.941315
6             so   6.556339
7            dae   6.288141
8   er_sgg_woman   4.204430
9            bid  10.293936
10           wgr   7.101599
11         cpp.1   4.433083
12          hbtp   6.735898
13           s10   8.012272


In [399]:
# target 열을 제외한 열 선택
data_vif = data[['target', 'ap', 'hos', 'popu_pgr', 'lta', 'ic', 'psap', 'so', 'dae', 'er_sgg_woman', 'bid', 'wgr',
                 'cpp.1', 'hbtp', 's10']]

data_vif_normalize = data_vif.columns[data_vif.columns != 'target']

# Min-Max 정규화 함수
normalize = lambda x: (x - x.min()) / (x.max() - x.min())

# 선택한 열에 대해 정규화 수행
data_vif[data_vif_normalize] = data_vif[data_vif_normalize].apply(normalize)
data_vif

,target,ap,hos,popu_pgr,lta,ic,psap,so,dae,er_sgg_woman,bid,wgr,cpp.1,hbtp,s10
0,1,0.245105,0.245283,0.317616,0.566492,0.303167,0.264538,0.484749,0.544075,0.347534,0.205128,0.364865,0.367347,0.283286,0.196998
1,0,0.035834,0.721698,0.049822,0.360731,0.133484,0.188141,0.006137,0.201758,0.008408,0.128205,0.378378,0.085714,0.549575,0.178236
2,1,0.138090,0.410377,0.179715,0.431393,0.185520,0.038769,0.075746,0.094313,0.095291,0.102564,0.189189,0.571429,0.249292,0.189493
3,0,0.051114,0.009434,0.078292,0.449108,0.169683,0.169897,0.022153,0.201758,0.066143,0.006410,0.074324,0.975510,0.430595,0.427767
4,1,0.482097,0.867925,0.326512,0.641543,0.079186,0.164196,0.901647,0.796907,0.799327,0.115385,0.310811,0.220408,0.359773,0.185741
5,1,0.183958,0.683962,0.266014,0.382396,0.110860,0.566705,0.000636,0.201758,0.993274,0.064103,0.344595,0.334694,0.300283,0.208255
6,0,0.018634,0.386792,0.092527,0.327136,0.235294,0.136830,0.224510,0.086801,0.015135,0.237179,0.236486,0.346939,0.181303,0.307692
7,1,0.360261,0.268868,0.250000,0.837450,0.156109,0.462942,0.980514,0.700980,0.240471,0.102564,0.412162,0.228571,0.305949,0.435272
9,1,0.420921,0.759434,0.007117,0.000000,0.009050,0.842645,0.238208,0.519015,0.243274,0.032051,0.189189,0.081633,0.351275,0.168856
10,1,0.396096,0.306604,0.173488,0.057814,0.033937,0.779932,0.138097,0.037620,0.243274,0.025641,0.290541,0.053061,0.337110,0.181989


'ap', 'hos', 'popu_pgr', 'lta', 'ic', 'psap', 'so', 'dae', 'er_sgg_woman', 'bid', 'wgr',
                 'cpp.1', 'hbtp', 's10'

In [415]:
# target 열을 제외한 열 선택
data_vif = data[['target', 'ap', 'hos', 'popu_pgr', 'lta']]

data_vif_normalize = data_vif.columns[data_vif.columns != 'target']

# Min-Max 정규화 함수
normalize = lambda x: (x - x.min()) / (x.max() - x.min())

# 선택한 열에 대해 정규화 수행
data_vif[data_vif_normalize] = data_vif[data_vif_normalize].apply(normalize)

# 로지스틱 회귀분석을 위한 독립변수와 종속변수 분리
X = data_vif.drop('target', axis=1)
y = data_vif['target']

# 상수(intercept) 열 추가
X = sm.add_constant(X)

# 로지스틱 회귀모델 학습
logit_model = sm.Logit(y, X)
result = logit_model.fit()

# 회귀분석 결과 요약
summary = result.summary()
print(summary)

# 후진소거법으로 제거

Optimization terminated successfully.
         Current function value: 0.268127
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                   64
Model:                          Logit   Df Residuals:                       59
Method:                           MLE   Df Model:                            4
Date:                Fri, 02 Jun 2023   Pseudo R-squ.:                  0.6030
Time:                        15:31:48   Log-Likelihood:                -17.160
converged:                       True   LL-Null:                       -43.230
Covariance Type:            nonrobust   LLR p-value:                 1.290e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.8804      2.099     -3.277      0.001     -10.995      -2.766
ap            12.6630      4.